# Kartik Spark Endpoint

In [ ]:
import findspark
findspark.init("/opt/spark-2.1.0-bin-cdh5.9.1/")
from pyspark.sql import SparkSession

#spark = SparkSession.builder.getOrCreate()

spark = SparkSession.builder.\
    config('spark.executor.memory', '5g').\
    config('spark.driver.memory', '5g').\
    config('spark.driver.maxResultSize', '3g').\
    config('spark.dynamicAllocation.maxExecutors', 20).\
getOrCreate()


# Rahul Endpoint

In [ ]:
import findspark
findspark.init('/opt/cloudera/parcels/SPARK2/lib/spark2/')

# import the library
import pyspark
from pyspark.sql import SparkSession
# for Spark 2.0, we have a unified entry point to the cluster
spark = SparkSession.builder.\
    getOrCreate()

# for previous versions, we can simulate SparkContext and SQLContext
sc = spark.sparkContext
sqlContext = spark

# Display information about current execution
spark.conf.get('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES')